In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from gensim.models import Doc2Vec
from collections import namedtuple
import gensim.utils
import re
import string
import requests
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim import similarities 

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [2]:
df = pd.read_csv(r'Hasil Crawl Halal MUI.csv')
df

,Unnamed: 0,id,no,nama,produsen,sertif,tanggal
0,0,109,21949,cyber park karawaci lt. 2 (head office),"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
1,1,121,21949,foodmart surabaya town square,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
2,2,95,21949,hypermart batam tanjung pinang,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
3,3,66,21949,hypermart bau bau,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
4,4,45,21949,hypermart ciputra world surabaya,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
...,...,...,...,...,...,...,...
401646,401646,4,12758,한상궁 delicious scorched brown rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401647,401647,2,12758,한상궁 delicous scorched barley and rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401648,401648,1,12758,한상궁 delicous scorched rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401649,401649,3,12758,한상궁 organic delicious scorched rice,"Hando food co., ltd",00100086820118,"25-february-2022"""


In [3]:
df = df.drop_duplicates(subset=['nama'], keep='last')
df

,Unnamed: 0,id,no,nama,produsen,sertif,tanggal
0,0,109,21949,cyber park karawaci lt. 2 (head office),"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
1,1,121,21949,foodmart surabaya town square,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
2,2,95,21949,hypermart batam tanjung pinang,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
3,3,66,21949,hypermart bau bau,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
4,4,45,21949,hypermart ciputra world surabaya,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
...,...,...,...,...,...,...,...
401646,401646,4,12758,한상궁 delicious scorched brown rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401647,401647,2,12758,한상궁 delicous scorched barley and rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401648,401648,1,12758,한상궁 delicous scorched rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401649,401649,3,12758,한상궁 organic delicious scorched rice,"Hando food co., ltd",00100086820118,"25-february-2022"""


In [4]:
@st.cache
def get_data():
    return pd.read_csv("halalmui2.csv" ,sep='\t', quotechar="'")

def cleandata(kalimat):
    kalimatbaru=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', kalimat)
    kalimatbaru=stopword.remove(kalimatbaru)
    return kalimatbaru

def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [5]:
lawdata=df
tanya = np.array(df.nama)
jawab=np.array(df.nama+',Produsen '+df.produsen)
data=np.array(df.nama+df.produsen)

2021-04-29 16:15:35.512 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [6]:
def get_response(q):
	highsim=0.0
	highnum=0
	blank=0    
	for i,line in enumerate(data):
		if (type(line)==str):
			qlist=cleandata(q).lower().split()
			linelist=cleandata(line).lower().split()
			simnow=jaccard_similarity(qlist,linelist)
			if(simnow>highsim):
				highnum=i
				highsim=simnow
				##if(highsim<0.2):
					##highnum=236660
	return(highnum)  
	

In [7]:
def get_response1(q):
	highsim=0.0
	highnum=0
	for i,line in enumerate(data):
		if (type(line)==str):
			qlist=cleandata(q).lower().split()
			linelist=cleandata(line).lower().split()
			simnow=jaccard_similarity(qlist,linelist)
			if(simnow>highsim):
				highnum=i
				highsim=simnow
	return(highsim)  
	

In [8]:
st.title("Assalamu 'alaikum di Open data konstitusi :)")

cari = st.text_input("Masukkan pertanyaan", "Apa tugas dari lembaga negara?")
  
# display the name when the submit button is clicked
# .title() is used to get the input text string 
if(st.button('Submit')):
	result = cari.title().lower()
	if(result):
		has=get_response(result)
		st.markdown(jawab[has])
	else:
		st.error("Masukkan input ya")

2021-04-29 16:15:36.632 WARNING root: 
  command:

    streamlit run C:\Users\LENOVO\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [9]:
docs2 = pd.read_csv('mergebersihindo.csv')
docs2 = docs2.drop_duplicates(subset=['nama_produk'], keep='last')
docs2 = docs2.drop('Unnamed: 0', 1)
docs2

,komposisi,nama_produk
1,NaN,Pristine 8+ Air Mineral
2,NaN,Crystalline Air Mineral
4,"Calcium, magnesium, sodium, potassium, nitrate...",Le Minerale Air Mineral
7,Air mineral,Club Air Mineral
8,NaN,Aqua Galon Kosong + Isi Ulang
...,...,...
1327,"Air, keju cheddar, pengental pati modifikasi a...",Pro Chiz Keju Gold
1329,biji bunga canola.BPOM RI ML 210216001071Diimp...,Dougo Canola Oil
1330,"Minyak nabati, air, garam, pengemulsi monodigl...",Blue Band Margarine Serbaguna
1331,NaN,Blue Band Margarine Cake & Cookie


In [10]:
docs3 = docs2.iloc[:3]
docs3

,komposisi,nama_produk
1,NaN,Pristine 8+ Air Mineral
2,NaN,Crystalline Air Mineral
4,"Calcium, magnesium, sodium, potassium, nitrate...",Le Minerale Air Mineral


In [11]:
items = []

In [12]:
jawab[get_response("Astor Wafer Stick Chocolate 150G")]

'Wafer stick astor coklat,Produsen Pt. mayora indah tbk'

In [13]:
for i, row in docs2.iterrows():
    q1 = row['nama_produk']
    q2 = str(get_response1(q1))
    items.append(q1+"|"+jawab[get_response(q1)]+"|"+q2)

In [14]:
get_response1("Astor Wafer Stick Chocolate 150G")

0.3333333333333333

In [15]:
print(items)

['Pristine 8+ Air Mineral|Air mineral botol aqua,Produsen Kampung kecil|0.3333333333333333', 'Crystalline Air Mineral|Air mineral botol aqua,Produsen Kampung kecil|0.4', 'Le Minerale Air Mineral|Air mineral le minerale,Produsen Cv. oto boga jaya|0.375', 'Club Air Mineral|Air mineral botol aqua,Produsen Kampung kecil|0.4', 'Aqua Galon Kosong + Isi Ulang|Air isi ulang panji galon,Produsen Panji galon|0.3333333333333333', 'Aqua Air Mineral Click N Go Kids|Air mineral merek aqua 240 ml,Produsen Sego pecel bu broto|0.23076923076923078', 'Aqua Air Mineral Click N Go|Air mineral botol aqua,Produsen Kampung kecil|0.25', 'Nestle Air Mineral Pure Life|Air mineral nestle pure life,Produsen Pt. kuliner sukses inovatif|0.4444444444444444', 'Ades Air Mineral|Air mineral botol aqua,Produsen Kampung kecil|0.4', 'Aqua Galon|Aqua botol,Produsen Drchicken sumbar|0.25', 'Aqua Air Mineral|Air mineral botol aqua,Produsen Kampung kecil|0.4', 'Indomaret Air Oksi|Indomaret  - air mineral,Produsen Pt. sariguna 

In [16]:
from pandas import DataFrame
df2 = DataFrame (items,columns=['produk'])
df2.head()

,produk
0,Pristine 8+ Air Mineral|Air mineral botol aqua...
1,Crystalline Air Mineral|Air mineral botol aqua...
2,Le Minerale Air Mineral|Air mineral le mineral...
3,"Club Air Mineral|Air mineral botol aqua,Produs..."
4,Aqua Galon Kosong + Isi Ulang|Air isi ulang pa...


In [17]:
df2.to_csv(r'indomaretxhalalmuijaccardmerge1.csv')